In [15]:
import pandas as pd
import yfinance as yf
import datetime

from requests_ratelimiter import LimiterSession, RequestRate, Limiter, Duration

open_threshold = 0.75
close_threshold = 0.25

In [16]:
data = [
    ['FIS', 'FITB', 2.0166827274862578, -3.1511109715915553, 6.9597648631874955],
    ['NVR', 'SHW', 24.272809334325157, -1060.4473553353364, 411.0716827760592],
    ['ANET', 'KKR', 0.7256367325304199, 1.709350418853501, 5.850847056252798],
    ['AXP', 'BSX', 3.1432737857287134, -30.12418201008268, 18.848070994096062],
    ['DLR', 'PNR', 1.7594553702250173, -4.070780723144448, 5.885738451992167],
    ['PHM', 'RSG', 0.5809289971147701, -32.522091052644015, 13.12831723615651],
    ['MSI', 'WELL', 3.6456087790516287, -99.40166493362965, 47.540380960504955]
]
pairs = pd.DataFrame(data, columns=['ticker1', 'ticker2', 'hedge_ratio', 'spread_mean', 'spread_std'])
pairs

,ticker1,ticker2,hedge_ratio,spread_mean,spread_std
0,FIS,FITB,2.016683,-3.151111,6.959765
1,NVR,SHW,24.272809,-1060.447355,411.071683
2,ANET,KKR,0.725637,1.709350,5.850847
3,AXP,BSX,3.143274,-30.124182,18.848071
4,DLR,PNR,1.759455,-4.070781,5.885738
5,PHM,RSG,0.580929,-32.522091,13.128317
6,MSI,WELL,3.645609,-99.401665,47.540381


In [17]:
tickers = set(list(pairs['ticker1']))
tickers.update(list(pairs['ticker2']))
tickers

{'ANET',
 'AXP',
 'BSX',
 'DLR',
 'FIS',
 'FITB',
 'KKR',
 'MSI',
 'NVR',
 'PHM',
 'PNR',
 'RSG',
 'SHW',
 'WELL'}

In [18]:
# Get today's date and yesterday's date
today = datetime.datetime.now().date()
yesterday = today - datetime.timedelta(days=7)  # Get a week back to ensure we cover weekends/holidays
day_before = yesterday - datetime.timedelta(days=1)

history_rate = RequestRate(1, Duration.SECOND)
limiter = Limiter(history_rate)
session = LimiterSession(limiter=limiter)

session.headers['User-agent'] = 'tickerpick/1.1'

# Download historical data for AAPL
data = yf.download(list(tickers), start=yesterday, end=today, auto_adjust=False, session=session)

# # Get the last available close price
# last_close = data['Adj Close'][-1]
# print(f"Last trading day's close price for AAPL: {last_close}")

[*********************100%***********************]  14 of 14 completed


In [19]:
last_day_price = data['Adj Close'].iloc[-1]
last_day_price

Ticker
ANET     109.779999
AXP      304.140015
BSX      103.160004
DLR      179.339996
FIS       81.639999
FITB      42.639999
KKR      144.970001
MSI      421.269989
NVR     7909.520020
PHM      121.169998
PNR      104.870003
RSG      242.309998
SHW      340.070007
WELL     161.089996
Name: 2025-07-22 00:00:00, dtype: float64

Теперь на основе цены закрытия предыдущего дня определить, стоит ли открывать позицию сегодня.

In [20]:
def calculate_positions(row, prices, open_threshold, close_threshold):
    ticker1 = row['ticker1']
    ticker2 = row['ticker2']
    hedge_ration = row['hedge_ratio']
    spread_mean = row['spread_mean']
    spread_std = row['spread_std']

    spread = prices[ticker1] - hedge_ration * prices[ticker2]
    
    zscore = (spread - spread_mean) / spread_std

    if (zscore >= open_threshold):
        return pd.Series(['short', 'long'])
    elif (zscore <= -open_threshold):
        return pd.Series(['long', 'short'])
    elif (zscore <= close_threshold):
        return pd.Series(['close', 'close'])
    elif (zscore >= -close_threshold):
        return pd.Series(['close', 'close'])
    else:
        return pd.Series(['continue_prev', 'continue_prev'])


pairs[['pos_ticker1', 'pos_ticker2']] = pairs.apply(lambda row: calculate_positions(row, last_day_price, open_threshold, close_threshold), axis=1)
pairs

,ticker1,ticker2,hedge_ratio,spread_mean,spread_std,pos_ticker1,pos_ticker2
0,FIS,FITB,2.016683,-3.151111,6.959765,close,close
1,NVR,SHW,24.272809,-1060.447355,411.071683,short,long
2,ANET,KKR,0.725637,1.709350,5.850847,close,close
3,AXP,BSX,3.143274,-30.124182,18.848071,close,close
4,DLR,PNR,1.759455,-4.070781,5.885738,close,close
5,PHM,RSG,0.580929,-32.522091,13.128317,short,long
6,MSI,WELL,3.645609,-99.401665,47.540381,long,short


NYSE работает с 15:30 до 22:00 по Амстердаму
- 22.07.2025 - Открыл шорты по ANET и WELL, лонги по MSI и KKR
- 23.07.2025
    - Нужно закрыть шорт ANET и лонг kKR
    - Нужно открыть шорт NVR и лонг SHW
    - Нужно открыть шорт PHM и лонг RSG
- 24.07.2025